In [1]:
import os
import sys
import numpy as np
import joblib
from pathlib import Path
from dataclasses import dataclass
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PROJECT_FOLDER"))
from src.utils.common import logger
from src.data.data_transformation import DataTransformation
from src.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    source_path: Path
    cleaned_data_path: Path
    transformed_data_path: Path
    train_data_path: Path
    test_data_path: Path

@dataclass(frozen=True)
class ModelPredictorConfig:
    root_dir: Path
    model_path: Path 

In [2]:
os.chdir("..")

In [3]:
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
        schema_filepath: str = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self):
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            source_path=config.source_path,
            cleaned_data_path=config.cleaned_data_path,
            transformed_data_path=config.transformed_data_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
        )
        
        return data_transformation_config
    
    def get_model_predictor_config(self):
        """
        Get configuration for model training
        """
        config = self.config.model_evaluation

        create_directories([config.root_dir])
        
        model_predictor_config = ModelPredictorConfig(
            root_dir = Path(config.root_dir),
            model_path = Path(config.model_path),
        )
        
        return model_predictor_config

In [4]:
class ModelPredictor:
    def __init__(self, config: ModelPredictorConfig):
        self.config = config
        self.model = self.get_model()
        
    def get_model(self):
        logger.info("Load model")
        model = None        
        with open(self.config.model_path, 'rb') as f:
            model = joblib.load(f)
        return model
    
    def predict(self, data: np.array) -> np.array:
        """
        Predict the data
        
        Args:
            data (np.array): Preprocessed text
        Return:
            np.array: Prediction results
        """
        logger.info("Predict")
        prediction = self.model.predict(data)
        return prediction

In [5]:
try:
    configuration_manager = ConfigurationManager()
    data_transformation = DataTransformation(configuration_manager.get_data_transformation_config())
    model_predictor = ModelPredictor(configuration_manager.get_model_predictor_config())
    data = input("Review: ")
    preprocessed_data = data_transformation.preprocess_text(data)
    preprocessed_data = np.array([preprocessed_data])
    prediction = model_predictor.predict(preprocessed_data)
    if prediction == 1:
        print("Positive")
    if prediction == 0:
        print("Negative")
except Exception as e:
    logger.error(e)

2024-03-13 15:24:41,478 - sentiment-classifier-logger - INFO - yaml file: config\config.yaml loaded successfully
2024-03-13 15:24:41,480 - sentiment-classifier-logger - INFO - yaml file: params.yaml loaded successfully
2024-03-13 15:24:41,484 - sentiment-classifier-logger - INFO - yaml file: schema.yaml loaded successfully
2024-03-13 15:24:41,485 - sentiment-classifier-logger - INFO - Created directory at: artifacts
2024-03-13 15:24:41,487 - sentiment-classifier-logger - INFO - Created directory at: artifacts/data_transformation
2024-03-13 15:24:41,549 - sentiment-classifier-logger - INFO - Created directory at: artifacts/model_evaluation
2024-03-13 15:24:41,551 - sentiment-classifier-logger - INFO - Load model
2024-03-13 15:24:51,223 - sentiment-classifier-logger - INFO - Predict
Positive
